In [6]:
import sys
import os
import pydicom
from pathlib import Path
import scipy.io


import matplotlib.pyplot as plt
import time
import pandas as pd
import csv
import math
import numpy as np

from shutil import copyfile

In [7]:
pythonCodePATH = '../../MEDomicsLab-develop/Code'
sys.path.append(pythonCodePATH)
from MEDimage.MEDimage import MEDimage

# MIM export folder is '/data/public/MIM_BMETS_V6/export_20191207'
# Each directory should contain image scan and rtstruct dicoms

In [10]:
images_dir = '/data/public/MIM_BMETS_V6/0_mim_export/'#'/data/public/MIM_BMETS_V6/0_mim_export'

dir_list = os.listdir(images_dir)
if '.DS_Store' in dir_list:
        dir_list.remove('.DS_Store')
dir_list.sort()

all_dirs = iter(dir_list)


In [11]:
len(dir_list)

2138

# Create a MEDImage obj for each directory and combine masks of all mets
# Save MEDImage obj

In [17]:
totStudies = 0
allMetsROIs = []
totMets = 0
#all_dirs = ['2015-06_2__Studies']#dirs # skipped 2011-03__Studies
#all_dirs = ['1997-07__Studies']

notMetsROIs = ['{Brain}','{Ventricles}','{*Skull}','{optic}','{motor}','{cord}','{External}','{ISOCENTER}', \
             '{avoid}','{Fornix}','{optic_n}','{brain_4gy}','{brain_4GY}','{l-motor}','{brainstem}', \
             '{intcaps}','{ON}','{cyst3}','{olfact}','{chiasm}','{cavity}','{L_optic_n}', \
             '{Avoidance}','{opticpath}','{right_hc}','{old_set}','{test}','{ROI}', \
             '{avoid motor}','{Avoid}','{L optic n}','{14&16}','{2,3,4}']#,'{Target 7}','{Target 6}']

for each_dir in dir_list[804:805]:
    print('Reading in dir: ',each_dir)
    
    # read dir
    dirFiles = os.listdir(images_dir + '/' + each_dir)
    if '.DS_Store' in dirFiles:
        dirFiles.remove('.DS_Store')
    dirFiles.sort()


    # read RTStruct
    f = pydicom.read_file(images_dir + '/' + each_dir + '/' + dirFiles[-1], force=True)    
    # StructureSetROISequence, number of ROIs
    structROIs = f.StructureSetROISequence
    nROIs = int(structROIs[-1].ROINumber)
    #print('Number of rois: ', structROIs[-1].ROINumber)
    
    # go through each ROI and sum masks of mets a created 'roiComb' with list of roi indexes to combine
    s = 0
    roiComb = ''
    for j in range(nROIs):
        # get roi name and id

        #print(rtPath)
        roi = '{' + structROIs[j].ROIName + '}'
        roiN = structROIs[j].ROINumber

        # used for brain mask
#         if roi == '{Brain}':
#             roiComb = str(roiN -1)
#             break
        
        # skip roi if it is not a targeted met
        if roi in notMetsROIs or roi[1]=='*':
            continue
        # sum mets rois into 1 mask (MASKSdata)
        else:
            allMetsROIs.append(roi) # save list of ROIs for review
            totMets = totMets + 1

            #print(j, "reading the following roi:", roi )
            if roiComb=='':
                roiComb = str(roiN -1)
            else:
                roiComb = roiComb + '+' + str(roiN -1)

    rtPath = Path(images_dir + '/' + each_dir)
    print('ROIs:',roiComb)
    MEDIobj = MEDimage(path_patient=rtPath, name_roi=roiComb, compute_radiomics_features=False, save_modified_image=True)

    # save object
    outputDir = '/data/public/MIM_BMETS_V6/1_medimage_mets/'
    study_name = f.PatientID + '_' + f.StudyDate 
    MEDIobj.save_MEDimage_object(path_save=outputDir,name_save=study_name,save_radiomics_only=False)
    print('Save MEDImage: ', study_name);

    

Reading in dir:  BrainMets-UCSF-01056_20080109
0 reading the following roi: {*Skull}
1 reading the following roi: {Brain}
2 reading the following roi: {Ventricles}
ROIs: 
-> IMAGE SUCCESSFULLY LOADED HERE

-> TESTING

-> LOAD TIME: 0.6350727081298828 seconds

Save MEDImage:  BrainMets-UCSF-01056_20080109
